This notebook generates the designs from "[Dispersion-engineered metasurfaces reaching broadband 90% relative diffraction efficiency](https://www.nature.com/articles/s41467-023-38185-2)" by Chen et al.

The detailed layout and dimensions for the meta-atoms are in the [supplementary material](https://static-content.springer.com/esm/art%3A10.1038%2Fs41467-023-38185-2/MediaObjects/41467_2023_38185_MOESM1_ESM.pdf).

In [ ]:
import jax
import jax.numpy as jnp
import numpy as onp
import matplotlib.pyplot as plt
from invrs_gym.challenges import meta_atom_library
from invrs_gym.utils import transforms
from totypes import json_utils
from skimage import measure


challenge = meta_atom_library()
params = challenge.component.init(jax.random.PRNGKey(0))


def plus_nanostructure(x1, x2, y1, y2, dim):
    """Return density array for plus-shaped nanostructures."""
    x, y = onp.meshgrid(
        onp.arange(-dim // 2, dim // 2) + 0.5,
        onp.arange(-dim // 2, dim // 2) + 0.5,
        indexing="ij",
    )
    p = onp.ones(x.shape, dtype=bool)
    p &= (x < x1 / 2) & (x > -x1 / 2)
    p &= (y < y1 / 2) & (y > -y1 / 2)
    p &= (x < x2 / 2) & (x > -x2 / 2) | ((y < y2 / 2) & (y > -y2 / 2))
    return p.astype(float)

def ibeam_nanostructure(x1, x2, y1, y2, dim, rotate):
    """Return density array for ibeam-shaped nanostructures."""
    x, y = onp.meshgrid(
        onp.arange(-dim // 2, dim // 2) + 0.5,
        onp.arange(-dim // 2, dim // 2) + 0.5,
        indexing="ij",
    )
    p = onp.ones(x.shape, dtype=bool)
    p &= (x < x1 / 2) & (x > -x1 / 2)
    p &= (y < y1 / 2) & (y > -y1 / 2)
    p &= (x > x2 / 2) | (x < -x2 / 2) | (y < (y1 / 2 - y2)) & (y > -(y1 / 2 - y2))
    if rotate:
        p = onp.rot90(p)
    return p.astype(float)

def two_rectangles(x1, x2, y1, y2, y3, dim, rotate):
    """Return density array for nanostructures consisting of two rectangles."""
    x, y = onp.meshgrid(
        onp.arange(-dim // 2, dim // 2) + 0.5,
        onp.arange(-dim // 2, dim // 2) + 0.5,
        indexing="ij",
    )
    ymid = (y1 - y3) / 2

    r1 = onp.ones(x.shape, dtype=bool)
    r1 &= (x < x1 / 2) & (x > -x1 / 2)
    r1 &= (y > -y2 / 2 - y1 + ymid) & (y < -y2 / 2 + ymid)

    r2 = onp.ones(x.shape, dtype=bool)
    r2 &= (x < x2 / 2) & (x > -x2 / 2)
    r2 &= (y < y2 / 2 + y3 + ymid) & (y > y2 / 2 + ymid)
    p = (r1 | r2).astype(float)

    if rotate:
        p = onp.rot90(p)
    return p.astype(float)

In [ ]:
# Create the density arrays for the 8 meta-atoms from the library. Values are
# taken from figure S3 of Chen et al.
nanostructures = (
    plus_nanostructure(x1=156, x2=86, y1=140, y2=86, dim=400),
    plus_nanostructure(x1=230, x2=60, y1=220, y2=60, dim=400),
    ibeam_nanostructure(x1=200, x2=80, y1=200, y2=60,dim=400, rotate=False),
    ibeam_nanostructure(x1=180, x2=60, y1=320, y2=120, dim=400, rotate=True),
    ibeam_nanostructure(x1=180, x2=60, y1=340, y2=120, dim=400, rotate=True),
    ibeam_nanostructure(x1=280, x2=80, y1=340, y2=130, dim=400, rotate=True),
    ibeam_nanostructure(x1=300, x2=60, y1=320, y2=110, dim=400, rotate=True),
    ibeam_nanostructure(x1=300, x2=60, y1=340, y2=90,dim=400, rotate=True),
)

# Plot the eight meta-atoms.
_, axs = plt.subplots(1, 8, figsize=(8, 2))
for i, ax in enumerate(axs.flatten()):
    im = ax.imshow(1 - nanostructures[i], cmap="gray")
    im.set_clim([-2, 1])
    for c in measure.find_contours(nanostructures[i]):
        ax.plot(c[:, 1], c[:, 0], "k", lw=1)
    ax.axis(False)
plt.subplots_adjust(hspace=0.1, wspace=0.1)

# Resample from 1 nm grid spacing to 5 nm grid spacing.
resampled_nanostructures = transforms.resample(jnp.stack(nanostructures), (8, 80, 80))
params["density"].array = resampled_nanostructures

with open("library1.json", "w") as f:
    f.write(json_utils.json_from_pytree(params))


In [ ]:
# Design D1 from supplementary figure 13.
nanostructures = (
    plus_nanostructure(x1=196, x2=76, y1=135, y2=50, dim=400),
    plus_nanostructure(x1=260, x2=76, y1=145, y2=50, dim=400),
    two_rectangles(x1=60, x2=200, y1=40, y2=100, y3=120, dim=400, rotate=False),
    ibeam_nanostructure(x1=290, x2=190, y1=230, y2=80, dim=400, rotate=False),
    ibeam_nanostructure(x1=330, x2=150, y1=190, y2=60, dim=400, rotate=False),
    ibeam_nanostructure(x1=290, x2=150, y1=250, y2=70, dim=400, rotate=False),
    ibeam_nanostructure(x1=310, x2=130, y1=250, y2=60, dim=400, rotate=False),
    ibeam_nanostructure(x1=330, x2=230, y1=330, y2=70, dim=400, rotate=False),
)

# Plot the eight meta-atoms.
_, axs = plt.subplots(1, 8, figsize=(8, 2))
for i, ax in enumerate(axs.flatten()):
    im = ax.imshow(1 - nanostructures[i], cmap="gray")
    im.set_clim([-2, 1])
    for c in measure.find_contours(nanostructures[i]):
        ax.plot(c[:, 1], c[:, 0], "k", lw=1)
    ax.axis(False)
plt.subplots_adjust(hspace=0.1, wspace=0.1)

# Resample from 1 nm grid spacing to 5 nm grid spacing.
resampled_nanostructures = transforms.resample(jnp.stack(nanostructures), (8, 80, 80))
params["density"].array = resampled_nanostructures

with open("design_d1.json", "w") as f:
    f.write(json_utils.json_from_pytree(params))

In [ ]:
# Design D2 from supplementary figure 13.
nanostructures = (
    ibeam_nanostructure(x1=300, x2=180, y1=280, y2=60, dim=400, rotate=True),
    ibeam_nanostructure(x1=340, x2=220, y1=320, y2=70, dim=400, rotate=True),
    plus_nanostructure(x1=130, x2=60, y1=170, y2=110, dim=400),
    plus_nanostructure(x1=170, x2=60, y1=205, y2=85, dim=400),
    two_rectangles(x1=190, x2=50, y1=130, y2=60, y3=50, dim=400, rotate=True),
    two_rectangles(x1=190, x2=170, y1=150, y2=90, y3=50, dim=400, rotate=True),
    two_rectangles(x1=230, x2=130, y1=150, y2=60, y3=70, dim=400, rotate=True),
    two_rectangles(x1=250, x2=250, y1=170, y2=90, y3=50, dim=400, rotate=True),
)

# Plot the eight meta-atoms.
_, axs = plt.subplots(1, 8, figsize=(8, 2))
for i, ax in enumerate(axs.flatten()):
    im = ax.imshow(1 - nanostructures[i], cmap="gray")
    im.set_clim([-2, 1])
    for c in measure.find_contours(nanostructures[i]):
        ax.plot(c[:, 1], c[:, 0], "k", lw=1)
    ax.axis(False)
plt.subplots_adjust(hspace=0.1, wspace=0.1)

# Resample from 1 nm grid spacing to 5 nm grid spacing.
resampled_nanostructures = transforms.resample(jnp.stack(nanostructures), (8, 80, 80))
params["density"].array = resampled_nanostructures

with open("design_d2.json", "w") as f:
    f.write(json_utils.json_from_pytree(params))

In [ ]:
# Design D5 from supplementary figure 13.
nanostructures = (
    ibeam_nanostructure(x1=320, x2=200, y1=300, y2=80,dim=400, rotate=False),
    ibeam_nanostructure(x1=340, x2=220, y1=300, y2=60, dim=400, rotate=False),
    ibeam_nanostructure(x1=340, x2=60, y1=320, y2=90,dim=400, rotate=False),
    plus_nanostructure(x1=196, x2=110, y1=130, y2=60, dim=400),
    plus_nanostructure(x1=296, x2=60, y1=205, y2=60, dim=400),
    plus_nanostructure(x1=296, x2=136, y1=155, y2=60, dim=400),
    ibeam_nanostructure(x1=240, x2=120, y1=240, y2=70, dim=400, rotate=False),
    two_rectangles(x1=230, x2=310, y1=50, y2=90, y3=150, dim=400, rotate=False),
)

# Plot the eight meta-atoms.
_, axs = plt.subplots(1, 8, figsize=(8, 2))
for i, ax in enumerate(axs.flatten()):
    im = ax.imshow(1 - nanostructures[i], cmap="gray")
    im.set_clim([-2, 1])
    for c in measure.find_contours(nanostructures[i]):
        ax.plot(c[:, 1], c[:, 0], "k", lw=1)
    ax.axis(False)
plt.subplots_adjust(hspace=0.1, wspace=0.1)

# Resample from 1 nm grid spacing to 5 nm grid spacing.
resampled_nanostructures = transforms.resample(jnp.stack(nanostructures), (8, 80, 80))
params["density"].array = resampled_nanostructures

with open("design_d5.json", "w") as f:
    f.write(json_utils.json_from_pytree(params))

In [ ]:
# Design D6 from supplementary figure 13.
nanostructures = (
    ibeam_nanostructure(x1=300, x2=180, y1=280, y2=60,dim=400, rotate=True),
    ibeam_nanostructure(x1=340, x2=220, y1=320, y2=70, dim=400, rotate=True),
    ibeam_nanostructure(x1=340, x2=60, y1=340, y2=80,dim=400, rotate=False),
    plus_nanostructure(x1=170, x2=60, y1=205, y2=85, dim=400),
    two_rectangles(x1=190, x2=50, y1=130, y2=60, y3=50, dim=400, rotate=True),
    two_rectangles(x1=190, x2=170, y1=150, y2=90, y3=50, dim=400, rotate=True),
    two_rectangles(x1=230, x2=130, y1=150, y2=60, y3=70, dim=400, rotate=True),
    two_rectangles(x1=250, x2=250, y1=170, y2=90, y3=50, dim=400, rotate=True),
)

# Plot the eight meta-atoms.
_, axs = plt.subplots(1, 8, figsize=(8, 2))
for i, ax in enumerate(axs.flatten()):
    im = ax.imshow(1 - nanostructures[i], cmap="gray")
    im.set_clim([-2, 1])
    for c in measure.find_contours(nanostructures[i]):
        ax.plot(c[:, 1], c[:, 0], "k", lw=1)
    ax.axis(False)
plt.subplots_adjust(hspace=0.1, wspace=0.1)

# Resample from 1 nm grid spacing to 5 nm grid spacing.
resampled_nanostructures = transforms.resample(jnp.stack(nanostructures), (8, 80, 80))
params["density"].array = resampled_nanostructures

with open("design_d6.json", "w") as f:
    f.write(json_utils.json_from_pytree(params))

In [ ]:
# Design D7 from supplementary figure 13.
nanostructures = (
    ibeam_nanostructure(x1=300, x2=180, y1=230, y2=80,dim=400, rotate=False),
    ibeam_nanostructure(x1=340, x2=140, y1=200, y2=60, dim=400, rotate=False),
    ibeam_nanostructure(x1=300, x2=140, y1=260, y2=70,dim=400, rotate=False),
    ibeam_nanostructure(x1=320, x2=120, y1=260, y2=60, dim=400, rotate=False),
    ibeam_nanostructure(x1=340, x2=220, y1=340, y2=70,dim=400, rotate=False),
    plus_nanostructure(x1=206, x2=86, y1=145, y2=60, dim=400),
    plus_nanostructure(x1=270, x2=86, y1=155, y2=60, dim=400),
    two_rectangles(x1=70, x2=210, y1=50, y2=90, y3=130, dim=400, rotate=False),
)

# Plot the eight meta-atoms.
_, axs = plt.subplots(1, 8, figsize=(8, 2))
for i, ax in enumerate(axs.flatten()):
    im = ax.imshow(1 - nanostructures[i], cmap="gray")
    im.set_clim([-2, 1])
    for c in measure.find_contours(nanostructures[i]):
        ax.plot(c[:, 1], c[:, 0], "k", lw=1)
    ax.axis(False)
plt.subplots_adjust(hspace=0.1, wspace=0.1)

# Resample from 1 nm grid spacing to 5 nm grid spacing.
resampled_nanostructures = transforms.resample(jnp.stack(nanostructures), (8, 80, 80))
params["density"].array = resampled_nanostructures

with open("design_d7.json", "w") as f:
    f.write(json_utils.json_from_pytree(params))